In [ ]:
# Import the relevant classes
from test_classes_EIS import DataHandler, RRCRCModel,RRCRCCPEModel, FitManager

# Example of how to fit multiple models with varying parameter numbers
data_handler = DataHandler('MPR files/1_00V_10cyc.mpr')

# Define initial guesses and bounds for each 
Rs_guess = 11.
R_guess = 1.5
fs_guess = 10**1.3
n_guess = 1.
R2_guess = 5e4
fs2_guess = 10**0.5
n2_guess = 1.

initial_guess_rrcrc = [Rs_guess, R_guess, 1e-6, R2_guess, 1e-6]
bounds_rrcrc = ([0, 0, 1e-9, 0, 1e-9], [100, 100, 1e-2, 1e7, 1e-2])

initial_guess_rrcrccpe = [Rs_guess, R_guess, fs_guess,n_guess, R2_guess, fs2_guess,n2_guess]
bounds_rrcrccpe = ([5, 1, 1e-2, 0.5, 1, 1e-2, 0.5], [15, 5, 1e7, 1.5, 1e6, 1e7, 1.5])

# Initialize models with dynamic number of parameters
rrcrc_model = RRCRCModel(initial_guess_rrcrc, bounds_rrcrc)
rcrc_model = RRCRCCPEModel(initial_guess_rrcrccpe, bounds_rrcrccpe)

# Initialize the fit manager
fit_manager = FitManager(data_handler)

# Fit both models
fitted_rrcrc_model, rrcrc_covariance = fit_manager.fit_model(rrcrc_model, fmin=0.1, fmax=1000)
fitted_rcrc_model, rcrc_covariance = fit_manager.fit_model(rcrc_model, fmin=0.1, fmax=1000)

# Print optimized parameters
print("Optimized RRCRC Model Parameters:", fitted_rrcrc_model.params)
print("Optimized RCRC Model Parameters:", fitted_rcrc_model.params)
